### Modules

In [1]:
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q wandb # for visualization
!pip install -q onnx # for saving weights

# for more details https://wandb.ai/site

     |████████████████████████████████| 1.8 MB 18.5 MB/s 
     |████████████████████████████████| 181 kB 62.7 MB/s 
     |████████████████████████████████| 157 kB 68.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 157 kB 70.4 MB/s 
     |████████████████████████████████| 157 kB 78.6 MB/s 
     |████████████████████████████████| 157 kB 81.2 MB/s 
     |████████████████████████████████| 157 kB 78.0 MB/s 
     |████████████████████████████████| 156 kB 78.6 MB/s 
     |████████████████████████████████| 13.1 MB 30.6 MB/s 


In [2]:
import requests
import json
import yaml
import glob
import os
import shutil
from sklearn.model_selection import train_test_split

In [3]:
def hagrid2yolo(bbox):
  return [bbox[0] + 0.5 * bbox[2], bbox[1] + 0.5 * bbox[3], bbox[2], bbox[3]]

### Dataset Downloading
### src: https://github.com/hukenovs/hagrid

In [4]:
res = requests.get('https://sc.link/AO5l')

if res.status_code == 200:
  with open('images.zip', 'wb') as f:
    f.write(res.content)
    print("Images successfully downloaded!")
else:
  print("Images couldn't downloaded!")

res = requests.get('https://sc.link/EQ5g')

if res.status_code == 200:
  with open('annotations.zip', 'wb') as f:
    f.write(res.content)
    print("Annotations successfully downloaded!")
else:
  print("Annotations couldn't downloaded!")

Images successfully downloaded!
Annotations successfully downloaded!


In [5]:
!mkdir images
!unzip -qq images.zip -d images
!rm images.zip

!mkdir annotations
!unzip -qq annotations.zip -d annotations
!rm annotations.zip

### Extracting the Bounding Boxes

In [6]:
# old dirs

old_img_dir = 'images'
old_ann_dir = 'annotations/ann_subsample'

## yolo-format dirs

root = 'dataset'
images = 'dataset/images'
labels = 'dataset/labels'

for path in [root, images, labels]:
  if not os.path.exists(path):
    os.mkdir(path)

for path in glob.glob(os.path.join(old_ann_dir, '*.json')):
  with open(path) as f:
    js = json.load(f)

    for key, val in js.items():
      with open(os.path.join(labels, f'{key}.txt'), 'w') as f:
        for bbox in val['bboxes']:
          f.write('0 {0} {1} {2} {3}\n'.format(*hagrid2yolo(bbox)))
      
      shutil.move(os.path.join(old_img_dir, path.split('/')[-1].replace('.json', ''), f'{key}.jpg'), images)

### Train-Valid-Test Split

In [7]:
train_dir, valid_dir, test_dir = 'dataset/train', 'dataset/valid', 'dataset/test'

for path in [train_dir, valid_dir, test_dir]:
  if not os.path.exists(path):
    os.mkdir(path)

imgs = glob.glob(os.path.join(images, '*.jpg'))

train_imgs, test_imgs = train_test_split(imgs, test_size=0.2, random_state=42)
train_imgs, valid_imgs = train_test_split(train_imgs, test_size=0.2, random_state=42)

In [8]:
for imgs, path in zip([train_imgs, valid_imgs, test_imgs], [train_dir, valid_dir, test_dir]):
  for img in imgs:
    label = img.replace(images, labels).replace('jpg', 'txt')
    shutil.move(img, path)
    shutil.move(label, path)

In [9]:
!rm -r images annotations
!rmdir dataset/images dataset/labels

### Yolov5 Installing
### src: https://github.com/ultralytics/yolov5

In [10]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -q -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 11957, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 11957 (delta 0), reused 0 (delta 0), pack-reused 11953
Receiving objects: 100% (11957/11957), 12.47 MiB | 27.33 MiB/s, done.
Resolving deltas: 100% (8219/8219), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 5.2 MB/s 


In [11]:
data = [{
    'path': '../dataset',
    'train': 'train',
    'val': 'valid',
    'test': 'test',
    'nc': 1,
    'names': ['hand']
}]

with open('dataset.yaml', 'w') as f:
  yaml.dump_all(data, f)

### Training

In [13]:
!python train.py --img 640            \
                 --batch 32           \
                 --epochs 20          \
                 --data dataset.yaml  \
                 --weights yolov5m.pt

wandb: Currently logged in as: farukcan. Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-46-g06831aa Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, 

### Evaluation

In [ ]:
!python val.py --data dataset.yaml                     \
               --weight runs/train/exp/weights/best.pt

val: data=dataset.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-308-g602d7ff Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning '/content/dataset/valid.cache' images and labels... 288 found, 0 missing, 0 empty, 0 corrupt: 100% 288/288 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 9/9 [00:17<00:00,  1.99s/it]
                 all        288        430      0.962      0.967      0.987      0.775
Speed: 0.2ms pre-process, 12.1ms inference, 1.3ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/val/exp3


### Inference

In [ ]:
!python detect.py --source ../dataset/test/                \
                  --weight runs/train/exp/weights/best.pt

detect: weights=['runs/train/exp/weights/best.pt'], source=../dataset/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-308-g602d7ff Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/360 /content/dataset/test/000dfde3-a3a2-41b3-a3eb-e52744bf3ac4.jpg: 640x480 1 hand, Done. (0.022s)
image 2/360 /content/dataset/test/000dfea0-f327-4193-b55c-c7111f118245.jpg: 640x480 1 hand, Done. (0.022s)
image 3/360 /content/dataset/test/0014f118-ac28-4438-a72e-01b185dafc72.jpg: 640x480 2 hands, Done. (0.022s)
image 4/360 /content/dataset/test/001e95

## Export

In [ ]:
!python export.py --weights yolov7-tiny.pt \
                  --simplify               \
                  --topk-all 100           \
                  --iou-thres 0.65         \
                  --conf-thres 0.35        \
                  --imgsz 640              

### Yolov7 Installing
### src: https://github.com/WongKinYiu/yolov7

In [10]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -q -r requirements.txt

Cloning into 'yolov7'...
remote: Enumerating objects: 749, done.
remote: Total 749 (delta 0), reused 0 (delta 0), pack-reused 749
Receiving objects: 100% (749/749), 67.46 MiB | 15.93 MiB/s, done.
Resolving deltas: 100% (375/375), done.
/content/yolov7
     |████████████████████████████████| 1.6 MB 35.9 MB/s 


In [11]:
data = [{
    'train': '../dataset/train',
    'val': '../dataset/valid',
    'test': '../dataset/test',
    'nc': 1,
    'names': ['hand']
}]

with open('data/dataset.yaml', 'w') as f:
  yaml.dump_all(data, f)

## Training

In [13]:
!python train.py --data data/dataset.yaml           \
                 --weights yolov7.pt                \
                 --img 640                          \
                 --epochs 20                        \
                 --batch-size 32                    \
                 --device 0

YOLOR 🚀 v0.1-105-g064c71e torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=32, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7.yaml', data='data/dataset.yaml', device='0', entity=None, epochs=20, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.custom.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp2', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=32, upload_dataset=False, weights='yolov7.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, bo

## Testing

In [ ]:
!python test.py --data data/dataset.yaml                  \
                --img 640                                 \
                --batch 32                                \
                --conf 0.001                              \
                --iou 0.65                                \
                --device 0                                \
                --weights runs/train/exp3/weights/best.pt

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data/dataset.yaml', device='0', exist_ok=False, img_size=640, iou_thres=0.65, name='exp', no_trace=False, project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['runs/train/exp3/weights/best.pt'])
YOLOR 🚀 v0.1-105-g064c71e torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model i

## Inference

In [ ]:
!python detect.py --weights yolov7.pt     \
                  --conf 0.25             \
                  --img-size 640          \
                  --source inference/

## Export

In [ ]:
!python export.py --weights yolov7-tiny.pt \
                  --grid                   \
                  --end2end                \
                  --simplify               \
                  --topk-all 100           \
                  --iou-thres 0.65         \
                  --conf-thres 0.35        \
                  --img-size 640           \
                  --max-wh 640         

Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(batch_size=1, conf_thres=0.35, device='cpu', dynamic=False, dynamic_batch=False, end2end=True, fp16=False, grid=True, img_size=[640, 640], include_nms=False, int8=False, iou_thres=0.65, max_wh=640, simplify=True, topk_all=100, weights='yolov7-tiny.pt')
YOLOR 🚀 v0.1-105-g064c71e torch 1.12.1+cu113 CPU

Fusing layers... 
Model Summary: 200 layers, 6219709 parameters, 6219709 gradients
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Starting TorchScript export with torch 1.12.1+cu113...
/content/yolov7/models/yolo.py:52: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Pyth